In [8]:
# Імпортуємо необхідні бібліотеки
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn import svm

In [9]:
class CustomKnn:
    def __init__(self, k: int) -> None:
        self.__k = k
    def fit(self, x_train, y_train):
        self.__x_train = x_train
        self.__y_train = y_train


    def euclidean_distance(self, point1, point2):
        return np.sqrt(np.sum((point1 - point2)**2))
    
    def pred(self, x_test):
        return np.array([self.__pred_one_point(point) for point in np.array(x_test)])

    
    def __pred_one_point(self, point):
        distances = [] # (i, distance)

        for i, x_point in enumerate(np.array(self.__x_train)):
            distances.append((i, self.euclidean_distance(point, x_point), np.array(self.__y_train)[i]))

        k_nearest_points = sorted(distances, key=lambda x: x[1])[:self.__k]

        classes_list = [point[-1] for point in k_nearest_points]

        unique, counts = np.unique(classes_list, return_counts=True)
        
        max_occurence = np.max(counts)

        vote_winner = []

        for i, count in enumerate(counts):
            if count == max_occurence:
                vote_winner.append(unique[i])

        winners_distances = []

        if len(vote_winner) > 1:
        
            for winner in vote_winner:
                winners_distances.append(np.sum([point[1] for point in k_nearest_points if point[-1] == winner]))

            vote_winner = [vote_winner[np.argmin(winners_distances)]]

        vote_winner = vote_winner[0]

        return vote_winner
        


In [10]:

iris = pd.read_csv('IRIS.csv')

X = iris.iloc[:, :-1]  # Ознаки (всі стовпці, крім останнього)
Y = iris.iloc[:, -1]   # Цільова змінна (останній стовпець)


# Розділення даних на тренувальний і тестовий набори
X_train, X_test, y_train, y_test = train_test_split(X, Y)

# Ініціалізація моделі KNN
k = 3  # Кількість сусідів
knn = KNeighborsClassifier(n_neighbors=k)

# Навчання моделі на тренувальних даних
knn.fit(X_train, y_train)

# Прогнози на тестовому наборі
y_pred = knn.predict(X_test)

# Використання нашого класу k-nn
knn = CustomKnn(k)
knn.fit(X_train, y_train)

y_custom_pred = knn.pred(X_test)

# Оцінка результатів класифікації

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

accuracy_on_custom_pred = accuracy_score(y_test, y_custom_pred )
report_on_custom_pred = classification_report(y_test, y_custom_pred )

print("\n\nAccuracy on custom prediction:", accuracy_on_custom_pred)
print("Classification Report on custom prediction:\n", report_on_custom_pred)




Accuracy: 0.9736842105263158
Classification Report:
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        13
Iris-versicolor       1.00      0.92      0.96        12
 Iris-virginica       0.93      1.00      0.96        13

       accuracy                           0.97        38
      macro avg       0.98      0.97      0.97        38
   weighted avg       0.98      0.97      0.97        38



Accuracy on custom prediction: 0.9736842105263158
Classification Report on custom prediction:
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        13
Iris-versicolor       1.00      0.92      0.96        12
 Iris-virginica       0.93      1.00      0.96        13

       accuracy                           0.97        38
      macro avg       0.98      0.97      0.97        38
   weighted avg       0.98      0.97      0.97        38



In [11]:

class BinarySVM:
    def __init__(self, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.w = None
        self.b = None

    def fit(self, X, y):
        n_samples, n_features = X.shape

        y_ = np.where(y <= 0, -1, 1)

        self.w = np.random.randn(n_features)
        self.b = 0

        for _ in range(self.n_iters):
            for idx, x_i in enumerate(X):
                condition = y_[idx] * (np.dot(x_i, self.w) - self.b) >= 1
                if condition:
                    self.w -= self.lr * (2 * 1 / self.n_iters * self.w)
                else:
                    self.w -= self.lr * (2 * 1 / self.n_iters * self.w - np.dot(x_i, y_[idx]))
                    self.b -= self.lr * y_[idx]

    def predict(self, X):
        approx = np.dot(X, self.w) - self.b
        return np.sign(approx)


class SVM:
    def __init__(self, classes, learning_rate=0.01, n_iters=1000):
        self.lr = learning_rate
        self.n_iters = n_iters
        self.classes = classes
        self.classifiers = [(BinarySVM(learning_rate, n_iters), claster) for claster in classes]

    def fit(self, X, y):
        index = 0
        for i in self.classes:
            binary_y = np.where(y == i, 1, -1)
            self.classifiers[index][0].fit(X, binary_y)
            index+=1

    def predict(self, X):
        scores = np.zeros((X.shape[0], len(self.classes)))
        for i in range(len(self.classes)):
            scores[:, i] = self.classifiers[i][0].predict(X)
        return [self.classifiers[i][1] for i in np.argmax(scores, axis=1)]

In [12]:
svm = svm.SVC().fit(X_train, y_train)
y_svm_pred= svm.predict(X_test)

custom_svm = SVM(set(y_train._values))
custom_svm.fit(X_train._values, y_train._values)

y_custom_svm_pred =custom_svm.predict(X_test._values)

In [13]:
accuracy = accuracy_score(y_test, y_svm_pred)
report = classification_report(y_test, y_svm_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

accuracy_on_custom_pred = accuracy_score(y_test, y_custom_svm_pred )
report_on_custom_pred = classification_report(y_test, y_custom_svm_pred )

print("\n\nAccuracy on custom prediction:", accuracy_on_custom_pred)
print("Classification Report on custom prediction:\n", report_on_custom_pred)

Accuracy: 0.9210526315789473
Classification Report:
                  precision    recall  f1-score   support

    Iris-setosa       1.00      1.00      1.00        13
Iris-versicolor       0.85      0.92      0.88        12
 Iris-virginica       0.92      0.85      0.88        13

       accuracy                           0.92        38
      macro avg       0.92      0.92      0.92        38
   weighted avg       0.92      0.92      0.92        38



Accuracy on custom prediction: 0.8947368421052632
Classification Report on custom prediction:
                  precision    recall  f1-score   support

    Iris-setosa       1.00      0.92      0.96        13
Iris-versicolor       0.79      0.92      0.85        12
 Iris-virginica       0.92      0.85      0.88        13

       accuracy                           0.89        38
      macro avg       0.90      0.90      0.90        38
   weighted avg       0.90      0.89      0.90        38

